In [1]:
import os
os.environ['TF_XLA_FLAGS'] = ''
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import numpy as np
from numpy import loadtxt, savetxt, asarray
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.regularizers import l1, l2, L1L2

/var/folders/y6/pdkrkx0s06d2k0_qhmmrpf8c0000gn/T/ipykernel_58573/2503121050.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


First, we import the reduced dataset, which has 15 features per each entry, and split it into three different sets for training, testing and validation. The features that we will consider are the rise times, fall times and the widths of the pulses for 5 different sensor regions. [PArise,PBrise,PCrise,PDrise,PFrise,PAfall,PBfall,PCfall,PDfall,PFfall,PAwidth,PBwidth,PCwidth,PDwidth,PFwidth]. 

In [6]:
filename = '../data/reduced_dataset/reduced_data.csv'
org_train_df = pd.read_csv(filename)
org_feature_df = org_train_df.drop(['Row', 'y','PBstart','PCstart','PDstart','PFstart'], axis=1)  # remove ID and postions/targets 
all_features = org_feature_df.columns

Next, we normalize the features to have range between [0,1] using the `Standard Scalar` from `scikit` library.

In [7]:
x = np.array(org_feature_df)

feature_df_np = org_feature_df.to_numpy()
scaler = StandardScaler()
scaler.fit(feature_df_np)
scaled_feature_np = scaler.transform(feature_df_np)
feature_df = pd.DataFrame(data=scaled_feature_np, columns=all_features)

In [2]:
def return_hos_rmse(e):
    return e[2]

In [4]:
filename = '../data/reduced_dataset/reduced_data.csv'

dataset = loadtxt(filename,delimiter=',',skiprows=1)

org_train_df = pd.read_csv(filename)
org_feature_df = org_train_df.drop(['Row', 'y','PBstart','PCstart','PDstart','PFstart'], axis=1)  # remove ID and postions/targets 
#org_feature_df = org_train_df.drop(['Row', 'y','PAamp','PBamp','PCamp','PDamp','PFamp'], 1)
all_features = org_feature_df.columns



x = np.array(org_feature_df)

y = dataset[:,len(dataset[0])-1]

#---normalize features to be the range of [0,1]
feature_df_np = org_feature_df.to_numpy()
scaler = StandardScaler()
scaler.fit(feature_df_np)
scaled_feature_np = scaler.transform(feature_df_np)
feature_df = pd.DataFrame(data=scaled_feature_np, columns=all_features)

x = np.array(feature_df)

print(np.shape(x))

y = dataset[:,len(dataset[0])-1]

data = []
test = []

y_norm = -41.9

for i in range(len(x)):
    temp = []
    for j in range(len(x[i])):
        temp.append(x[i][j])
    temp.append(float(y[i]))
    temp_y = int(np.round(abs(y[i]),0))
    if temp_y==13 or temp_y==30 or temp_y==42:
        test.append(temp)
    else:
        data.append(temp)

data = np.array(data)

training, validation = train_test_split(data, test_size = 0.2, random_state = 42,stratify=data[:,-1])

testing = np.array(test)

training[:, [5, 1]] = training[:, [1, 5]]
training[:, [10, 2]] = training[:, [2, 10]]
training[:, [5, 3]] = training[:, [3, 5]]
training[:, [6, 4]] = training[:, [4, 6]]
training[:, [11, 5]] = training[:, [5, 11]]
training[:, [10, 6]] = training[:, [6, 10]]
training[:, [12, 8]] = training[:, [8, 12]]
training[:, [11, 9]] = training[:, [9, 11]]
training[:, [12, 10]] = training[:, [10, 12]]
training[:, [13, 11]] = training[:, [11, 13]]

validation[:, [5, 1]] = validation[:, [1, 5]]
validation[:, [10, 2]] = validation[:, [2, 10]]
validation[:, [5, 3]] = validation[:, [3, 5]]
validation[:, [6, 4]] = validation[:, [4, 6]]
validation[:, [11, 5]] = validation[:, [5, 11]]
validation[:, [10, 6]] = validation[:, [6, 10]]
validation[:, [12, 8]] = validation[:, [8, 12]]
validation[:, [11, 9]] = validation[:, [9, 11]]
validation[:, [12, 10]] = validation[:, [10, 12]]
validation[:, [13, 11]] = validation[:, [11, 13]]

testing[:, [5, 1]] = testing[:, [1, 5]]
testing[:, [10, 2]] = testing[:, [2, 10]]
testing[:, [5, 3]] = testing[:, [3, 5]]
testing[:, [6, 4]] = testing[:, [4, 6]]
testing[:, [11, 5]] = testing[:, [5, 11]]
testing[:, [10, 6]] = testing[:, [6, 10]]
testing[:, [12, 8]] = testing[:, [8, 12]]
testing[:, [11, 9]] = testing[:, [9, 11]]
testing[:, [12, 10]] = testing[:, [10, 12]]
testing[:, [13, 11]] = testing[:, [11, 13]]


np.random.shuffle(testing)
np.random.shuffle(training)
np.random.shuffle(validation)

print(np.shape(training),np.shape(validation),np.shape(testing))

savetxt('../data/processed_csv/conv_training_reduced.csv', asarray(training),fmt='%s', delimiter=',')
savetxt('../data/processed_csv/conv_validation_reduced.csv', asarray(validation),fmt='%s', delimiter=',')
savetxt('../data/processed_csv/conv_testing_reduced.csv', asarray(testing),fmt='%s', delimiter=',')

(7151, 15)
(4508, 16) (1128, 16) (1515, 16)


In [6]:
train_dataset = loadtxt('../data/processed_csv/conv_training_reduced.csv',delimiter=',')
val_dataset = loadtxt('../data/processed_csv/conv_validation_reduced.csv',delimiter=',')
test_dataset = loadtxt('../data/processed_csv/conv_testing_reduced.csv',delimiter=',')

In [11]:
X = train_dataset[:,0:len(train_dataset[0])-1]
y = train_dataset[:,len(train_dataset[0])-1:]

X_val = val_dataset[:,0:len(val_dataset[0])-1]
y_val = val_dataset[:,len(val_dataset[0])-1:]

X_test = test_dataset[:,0:len(test_dataset[0])-1]
y_test = test_dataset[:,len(test_dataset[0])-1:]

In [14]:
L1 = 0.1
L2 = 0.1

EPOCHS = 1
BATCH_SIZE = 50
INPUT_DIM = 15

In [16]:
results = []

min_rmse = 1000

for i in range(EPOCHS):

    #CNN
    #'''
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=INPUT_DIM, kernel_size=5, strides=5,input_shape=(INPUT_DIM,1),kernel_regularizer=l1(L1)))
    #model.add(tf.keras.layers.Dropout(dr))
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.GRU(100, return_sequences='false', kernel_regularizer=l1(L1)))
    #model.add(tf.keras.layers.Dropout(dr))
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(units=50, kernel_regularizer=l1(L1)))
    #model.add(tf.keras.layers.Dropout(dr))
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(units=25, kernel_regularizer=l1(L1)))
    #model.add(tf.keras.layers.Dropout(dr))
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(units=10, kernel_regularizer=l1(L1)))
    #model.add(tf.keras.layers.Dropout(dr))
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    #'''



    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.RootMeanSquaredError()])


    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=0,patience=1)
    history = model.fit(X,y,epochs=i,batch_size=BATCH_SIZE,verbose=0,validation_data=(X_val,y_val),callbacks=[callback])

    if i==0:
        print(model.summary())
        print('')
    #model.save('test_DNN.h5')

    train_mse, train_rmse = model.evaluate(X, y, batch_size=50,verbose=0)
    val_mse, val_rmse = model.evaluate(X_val, y_val, batch_size=50,verbose=0)
    test_mse, test_rmse = model.evaluate(X_test, y_test, batch_size=50,verbose=0)

    print(train_rmse,val_rmse,test_rmse)
    results.append((train_rmse,val_rmse,test_rmse))

    if test_rmse<min_rmse:
        model.save('CNN_FullNPA_L1RegES_noBN_Model.h5')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 3, 15)             90        
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 3, 15)             0         
                                                                 
 gru_2 (GRU)                 (None, 3, 100)            35100     
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 3, 100)            0         
                                                                 
 dense_8 (Dense)             (None, 3, 50)             5050      
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 3, 50)             0         
                                                                 
 dense_9 (Dense)             (None, 3, 25)            

: 

In [ ]:
results.sort(reverse=False,key=return_hos_rmse)
np.savetxt('CNN_FullNPA_L1RegES_noBN_results.csv', np.asarray(results),fmt='%s', delimiter=',')